In [1]:
from collections import Counter, defaultdict

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
import numpy as np
import pandas as pd
import os
import sys

import matplotlib.pyplot as plt

In [6]:
root_dir = '/h/zycluke/Project/adversarial_attack/checkpoint/exp_4_10/emnist'
os.listdir(root_dir)

['wrn_D16_W16_FGSM',
 'wrn_D16_W4_FAB',
 'wrn_D16_W8_PGD',
 'wrn_D16_W1_PGD',
 'wrn_D16_W2_FGSM',
 'wrn_D16_W16_PGD',
 'wrn_D16_W2_FAB',
 'wrn_D16_W4_FGSM',
 'wrn_D16_W4_PGD',
 'wrn_D16_W8_FAB',
 'wrn_D16_W1_FAB',
 'wrn_D16_W8_FGSM',
 'wrn_D16_W2_PGD',
 'wrn_D16_W16_FAB',
 'wrn_D16_W1_FGSM']

In [63]:
line_config = {
    'rtest_acc-0.001': dict(name='Attack-0.001', type="scatter", mode='lines+markers',
                            line=dict(color='rgb(31, 119, 180)', dash='solid'),
                            marker=dict(color='rgb(31, 119, 180)', symbol='square', size=8)),
    'rtest_acc-0.01': dict(name='Attack-0.01', type="scatter", mode='lines+markers',
                           line=dict(color='rgb(227, 119, 194)', dash='solid'),
                           marker=dict(color='rgb(227, 119, 194)', symbol='diamond', size=8)),
    'rtest_acc-0.03': dict(name='Attack-0.03', type="scatter", mode='lines+markers',
                           line=dict(color='rgb(44, 160, 44)', dash='solid'),
                           marker=dict(color='rgb(44, 160, 44)', symbol='x', size=8)),
    'rtest_acc-0.1': dict(name='Attack-0.1', type="scatter", mode='lines+markers',
                          line=dict(color='rgb(148, 103, 189)', dash='solid'),
                          marker=dict(color='rgb(148, 103, 189)', symbol='circle', size=8)),
    'rtest_acc-0.3': dict(name='Attack-0.3', type="scatter", mode='lines+markers',
                          line=dict(color='rgb(255, 127, 14)', dash='solid'),
                          marker=dict(color='rgb(255, 127, 14)', symbol='cross', size=8)),
    'rtest_acc-0.7': dict(name='Attack-0.7', type="scatter", mode='lines+markers',
                          line=dict(color='rgb(140, 86, 75)', dash='solid'),
                          marker=dict(color='rgb(140, 86, 75)', symbol='hexagon', size=8)),
    'Train Acc.': dict(name='Train-acc', type="scatter", mode='lines+markers',
                       line=dict(color='rgb(8, 191, 194)', dash='solid'),
                       marker=dict(color='rgb(8, 191, 194)', symbol='star-triangle-up', size=8)),
    'Valid Acc.': dict(name='Attack-0.0', type="scatter", mode='lines+markers', line=dict(color='rgb(214, 39, 40)', dash='solid'),
                        marker=dict(color='rgb(214, 39, 40)', symbol='star', size=8)),
}

In [55]:
def get_data_uncertainty(data_list, use_median=False, use_max=False, scale=False):
    '''
    Get data uncertainty for shaded traces.

    Parameters
    ----------
    data_list: :py:class:`list` of :py:class:`list`
        The y values for several predictors.
    use_median: :py:class:`bool`
        Use Mean or Median. Default to Mean.
    use_max: :py:class:`bool`
        Use Standard Deviation or Max and Min Value. Default to Standard Deviation.

    Returns
    -------

    '''
    y = []
    y_err = []
    high = []
    low = []
    for data in data_list:
        if scale:
            data = np.array(data) * 100
        if use_median:
            mean = np.nanmedian(data)
        else:
            mean = np.nanmean(data)
        y.append(mean)

        if use_max:
            up = np.nanmax(data)
            down = np.nanmin(data)
            y_err.append((up - down) / 2)
            high.append(up)
            low.append(down)
        else:
            std = np.nanstd(data)
            y_err.append(std)
            high.append(mean + std)
            low.append(mean - std)

    low = low[::-1]
    return y, y_err, high, low

In [54]:
def get_line_trace(x, y_list, name, showlegend=True):
    config = line_config[name]

    x_rev = x[::-1]
    y, y_err, high, low = get_data_uncertainty(y_list)

    traces = [
        dict(
            x=x,
            y=y,
            type=config['type'],
            line=config['line'],
            name=config['name'],
            #                 legendgroup = config['name'],
            showlegend=showlegend,
            text=['{0:.3f}±{1:.3f}'.format(a, b)
                  for a, b in zip(y, y_err)],
            marker=config['marker'],
        ),
        dict(
            x=x + x_rev,
            y=high + low,
            type=config['type'],
            line=dict(color='rgba(255,255,255,0)'),
            name=config['name'] + '_fill',
            #                 legendgroup = config['name'],
            hoverinfo='skip',
            fill='toself',
            fillcolor=config['line']['color'].replace('rgb', 'rgba').replace(')', ',0.2)'),
            showlegend=False,
        ),
    ]

    return traces

# PGD

In [50]:
def get_res_seed(exp_dir):
    df = pd.read_csv('{}/{}'.format(exp_dir,'train_result.csv'))
    col_acc = [name for name in df.columns if 'cc' in name and 'topk' not in name]
    acc_dict = df[col_acc].iloc[-1,:].to_dict()
    return acc_dict

In [42]:
def get_res_seeds(exp_dir):
    res = defaultdict(list)
    for seed in os.listdir(exp_dir):
        res_seed = get_res_seed('{}/{}'.format(exp_dir,seed))
        for k,v in res_seed.items():
            res[k].append(v)
    return res

In [43]:
def get_res_method(root_dir, name='PGD'):
    model_sizes = [1,2,4,8,16]
    res = defaultdict(list)
    for size in model_sizes:
        exp_dir = '{}/{}'.format(root_dir,'wrn_D16_W{}_{}'.format(size,name))
        res_model = get_res_seeds(exp_dir)
        for k,v in res_model.items():
            res[k].append(v)
    return res
    

In [84]:
def plot_res(root_dir, attack_method='PGD'):
    fig = make_subplots(rows=1, cols=1, subplot_titles=[attack_method])
    res = get_res_method(root_dir, name=attack_method)
    for k, v in res.items():
        if 'Acc' in k :
            continue
        traces = get_line_trace([1,2,4,8],res[k],name=k,showlegend=True)
        for trace in traces:
            fig.add_trace(trace, row=1, col=1)

    xaxis_layout = dict(
            title='Model Size',
            showgrid=True,
            showline=True,
            mirror=True,
            tickmode='array',
            ticks='outside',
            tickfont=dict(size=18),
            linecolor='rgba(0,0,0,1)',
            gridcolor='rgba(0,0,0,0.1)',
            showticklabels=True,
            titlefont=dict(size=20))

    yaxis_layout = dict(
        title='Test Accuracy',
        showgrid=True,
        showline=True,
        mirror=True,
        tickmode='array',
        ticks='outside',
        tickfont=dict(size=18),
#         tickvals=[10 * i for i in range(11)],
        linecolor='rgba(0,0,0,1)',
        gridcolor='rgba(0,0,0,0.1)',
        showticklabels=True,
        titlefont=dict(size=20))

    fig['layout']['xaxis'].update(xaxis_layout)
    fig['layout']['yaxis'].update(yaxis_layout)

    fig.update_layout(
            autosize=False,
            width=450,
            height=350,
            legend=dict(
                bgcolor='rgba(255,255,255,0.5)',
                bordercolor='rgba(0,0,0,0.2)',
                borderwidth=1,
                font=dict(size=13),
                yanchor="top",
                y=0.96,
                xanchor="right",
                x=0.96,
            ),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(255,255,255,0)',
            margin=dict(l=0, r=0, t=25, b=0),
            font=dict(family="Times")
        )

    fig.show()

In [85]:
plot_res(root_dir,attack_method='PGD')

/h/zycluke/miniconda3/envs/al/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning:

Mean of empty slice

/h/zycluke/miniconda3/envs/al/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning:

Degrees of freedom <= 0 for slice.



In [86]:
plot_res(root_dir,attack_method='FGSM')

In [87]:
plot_res(root_dir,attack_method='FAB')

/h/zycluke/miniconda3/envs/al/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning:

Mean of empty slice

/h/zycluke/miniconda3/envs/al/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning:

Degrees of freedom <= 0 for slice.

